# Segmenting and Clustering Neighborhoods in Toronto
----
## Question 3. Exploring and Clustering the Neighborhoods in Toronto

#### Importing necessary libraries

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Reading the dataframe we prepared in previous question from csv file into dataframe df

In [8]:
df=pd.read_csv('TorontoGeoData.csv')

In [9]:
df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [10]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [11]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Getting geographical coordinates of Toronto

In [12]:
toronto_add='Toronto, Ontario' #CITY ADDRESS
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(toronto_add)
latitude=location.latitude
longitude=location.longitude
print("The coordinates of Toronto are {},{}".format(latitude,longitude))

The coordinates of Toronto are 43.653963,-79.387207


#### Creating a map of Toronto city with all the postal codes

In [13]:
Toronto_map=folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postalcode,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['PostalCode'],df['Borough'],df['Neighborhood']):
    label='Postal Code:{},Neighborhoods:{},Borough:{}'.format(postalcode,neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],radius=5,color='blue',popup=label,
                        fill=True,fill_color='#3186cc',fill_opacity=0.7,
                        parse_html=True).add_to(Toronto_map)

In [14]:
Toronto_map

#### Defining foursquare credentials

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
import requests# library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize# tranform JSON file into a pandas dataframe

#### We create a function to extract dataframe of top 100 venues within 500m radius of each of the postal codes in Toronto

In [17]:
limit=100
def getNearbyVenues(codes,names,latitudes,longitudes,radius=500):
    venues_list=[]
    for code,name,lat,lng in zip(codes,names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(code,name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],
                          v['venue']['location']['lng'],
                          v['venue']['categories'][0]['name'])for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['PostalCode','Neighborhood','Neighborhood Latitude',
                          'Neighborhood Longitude','Venue name','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

#### Toronto Venues dataframe

In [18]:
Toronto_Venues=getNearbyVenues(codes=df['PostalCode'],
                               names=df['Neighborhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude'],radius=500)

In [19]:
Toronto_Venues.head()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue name,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### We check the shape of Toronto_Venues

In [20]:
Toronto_Venues.shape

(2258, 8)

#### Now, we check how many venues were returned for each postal code

In [21]:
Toronto_Venues.groupby('PostalCode').count()[['Venue name']]

,Venue name
PostalCode,
M1B,2
M1C,2
M1E,9
M1G,4
M1H,8
M1J,1
M1K,4
M1L,10
M1M,3


#### Now we check the number of unique venue categories that were returned

In [22]:
print('There are {} unique venue categories'.format(len(Toronto_Venues['Venue Category'].unique())))

There are 280 unique venue categories


#### Now we analyze each neighborhood by creating a dataframe with one hot encoding

In [23]:
Toronto_onehot=pd.get_dummies(Toronto_Venues['Venue Category'],prefix='',prefix_sep='')#one hot encoding
#Add Neighborhood and PostalCode column back to Toronto_onehot dataframe
Toronto_onehot['PostalCode']=Toronto_Venues['PostalCode']
columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[columns]
Toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Toronto_onehot.shape

(2258, 281)

In [25]:
X=Toronto_Venues['Neighborhood']

In [26]:
Toronto_onehot[-1]=X

In [27]:
Toronto_onehot=Toronto_onehot.rename({-1:'Neighborhoods'},axis=1)

In [28]:
Toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhoods
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village


In [29]:
col=[Toronto_onehot.columns[0]]+[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[1:-1])
Toronto_onehot=Toronto_onehot[col]

In [30]:
Toronto_onehot.head()

,PostalCode,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,Victoria Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,Victoria Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Let's group rows by postal code by taking the mean of the frequency of occurrence of each category

In [31]:
Toronto_grouped=Toronto_onehot.groupby(['PostalCode','Neighborhoods'],sort=False).mean().reset_index()

In [32]:
Toronto_grouped.head()

,PostalCode,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000,0.000000
1,M4A,Victoria Village,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000,0.000000
2,M5A,"Regent Park, Harbourfront",0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000,0.020833
3,M6A,"Lawrence Heights, Lawrence Manor",0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0625,0.0,0.0,0.000000,0.125,0.000000
4,M7A,Queen's Park,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.025641,0.0,0.0,0.0000,0.0,0.0,0.025641,0.000,0.025641


#### Now, we print each neighborhood along with top 5 venues for first 10 postal codes

In [33]:
num=5
for code,nbd in zip(Toronto_grouped['PostalCode'][0:10],Toronto_grouped['Neighborhoods'][0:10]):
    print('----'+code+':'+nbd+'----')
    temp=Toronto_grouped[Toronto_grouped['PostalCode']==code].T.reset_index()
    temp=temp.iloc[2:]
    temp.columns=['venue category','freq']
    temp['freq']=temp['freq'].astype(float)
    temp.sort_values(by=['freq'],ascending=False,inplace=True)
    temp=temp.round({'freq':2})
    print(temp.reset_index(drop=True).head(num))

----M3A:Parkwoods----
              venue category  freq
0       Fast Food Restaurant  0.33
1                       Park  0.33
2          Food & Drink Shop  0.33
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00
----M4A:Victoria Village----
          venue category  freq
0           Hockey Arena  0.25
1           Intersection  0.25
2            Coffee Shop  0.25
3  Portuguese Restaurant  0.25
4      Accessories Store  0.00
----M5A:Regent Park, Harbourfront----
   venue category  freq
0     Coffee Shop  0.17
1             Pub  0.06
2          Bakery  0.06
3            Park  0.06
4  Breakfast Spot  0.04
----M6A:Lawrence Heights, Lawrence Manor----
           venue category  freq
0          Clothing Store  0.25
1  Furniture / Home Store  0.19
2           Women's Store  0.12
3       Accessories Store  0.06
4             Coffee Shop  0.06
----M7A:Queen's Park----
        venue category  freq
0          Coffee Shop  0.23
1                 Park  0.05
2     Sushi Restaurant

#### We now define a function to sort values of the venue in descending order

In [34]:
def most_common_venues(row,num_of_values):
    row_sorted=row.iloc[2:].sort_values(ascending=False)
    return row_sorted.index.values[0:num_of_values]

#### We now create a dataframe to contain top 10 venues of each postal code

##### We first create the name of columns of such dataframe

In [35]:
num_of_values=10
indicators=['st','nd','rd']
columns=['PostalCode','Neighborhoods']
for ind in np.arange(num_of_values):
    try:
        columns.append('{}{} Most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))  

##### Initializing a dataframe

In [36]:
Toronto_top=pd.DataFrame(columns=columns)

##### Filling in the dataframe

In [37]:
Toronto_top['PostalCode']=Toronto_grouped['PostalCode']
Toronto_top['Neighborhoods']=Toronto_grouped['Neighborhoods']

In [38]:
for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_top.iloc[ind][2:]=most_common_venues(Toronto_grouped.iloc[ind],num_of_values)

In [39]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store
1,M4A,Victoria Village,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,M5A,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Yoga Studio
3,M6A,"Lawrence Heights, Lawrence Manor",Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Queen's Park,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint


#### Now we run K means clustering to cluster the similar postal codes/Neighborhoods

In [40]:
#The features needed to run  kmeans clustering is the frequency of venue category.
#since K Means does not consider categorical variables, we drop the PostalCode and Neighborhoods columns from the Toronto_grouped dataframe
Toronto_grouped.drop(['PostalCode','Neighborhoods'],axis=1,inplace=True)
Toronto_grouped.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000,0.000000
1,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000,0.000000
2,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000,0.020833
3,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0625,0.0,0.0,0.000000,0.125,0.000000
4,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.025641,0.0,0.0,0.0000,0.0,0.0,0.025641,0.000,0.025641


In [41]:
k=5 # number of clusters
k_means=KMeans(n_clusters=k,random_state=0)
k_means.fit(Toronto_grouped)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [42]:
k_means.labels_[0:10] #We check the cluster labels of the first 10 rows

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int32)

In [43]:
# Now, we add the cluster labels column to the Toronto_top dataframe 
Toronto_top.insert(0,' Cluster labels',k_means.labels_)

In [44]:
Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,2,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store
1,1,M4A,Victoria Village,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,1,M5A,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Yoga Studio
3,1,M6A,"Lawrence Heights, Lawrence Manor",Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,1,M7A,Queen's Park,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint


In [45]:
#Now we merge the dataframe Toronto_top with df to add the latitude and longitude of each postal code
Toronto_data=df
Toronto_top=Toronto_top.join(Toronto_data.set_index('PostalCode'),on=['PostalCode'])

In [46]:
Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,Victoria Village,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,North York,Victoria Village,43.725882,-79.315572
2,1,M5A,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Yoga Studio,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,1,M6A,"Lawrence Heights, Lawrence Manor",Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,1,M7A,Queen's Park,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Queen's Park,Queen's Park,43.662301,-79.389494


In [47]:
#We drop the Neighborhood column as it is duplicated as Neighborhoods column
Toronto_top.drop(['Neighborhood'],axis=1,inplace=True)

In [48]:
Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Borough,Latitude,Longitude
0,2,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,North York,43.753259,-79.329656
1,1,M4A,Victoria Village,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,North York,43.725882,-79.315572
2,1,M5A,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Yoga Studio,Downtown Toronto,43.654260,-79.360636
3,1,M6A,"Lawrence Heights, Lawrence Manor",Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant,North York,43.718518,-79.464763
4,1,M7A,Queen's Park,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Queen's Park,43.662301,-79.389494


#### Visualising the clusters

In [49]:
map_clusters=folium.Map([latitude,longitude],zoom_start=10)#Creating the map

In [50]:
x=np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [51]:
for code,nbd,lat,lng,clus in zip(Toronto_top['PostalCode'],Toronto_top['Neighborhoods'],Toronto_top['Latitude'],Toronto_top['Longitude'],Toronto_top[' Cluster labels']):
    label=folium.Popup('Postal Code:{};Neighborhoods:{};Cluster:{}'.format(code,nbd,clus),parse_html=True)
    folium.CircleMarker([lat,lng],popup=label,color=rainbow[clus-1],
                       fill=True,
                       fill_color=rainbow[clus-1],
                       fill_opacity=0.6,
                       radius=5).add_to(map_clusters)

In [52]:
map_clusters

#### Examining the clusters

##### Cluster 1: Cluster Label 0

In [53]:
Toronto_top.loc[Toronto_top[' Cluster labels']==0,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,0,M1J,Scarborough Village,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,0,M4T,"Summerhill East, Moore Park",Tennis Court,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


###### We observe that Cluster 1 has neighborhoods with Playground, Yoga Studio, Drugstore, Dim Sum Restaurant, Diner, Discount Store, Dog Run, Doner Restaurant, Donut Shp  in Top 10 venues with  the two restaurants share similar ranking

##### Cluster 2: Cluster Label 1

In [54]:
Toronto_top.loc[Toronto_top[' Cluster labels']==1,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,1,M4A,Victoria Village,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,1,M5A,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Yoga Studio
2,1,M6A,"Lawrence Heights, Lawrence Manor",Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
3,1,M7A,Queen's Park,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
4,1,M1B,"Rouge, Malvern",Fast Food Restaurant,Print Shop,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
5,1,M3B,Don Mills North,Café,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Baseball Field,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
6,1,M4B,"Woodbine Gardens, Parkview Hill",Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Pharmacy,Café,Breakfast Spot,Bank
7,1,M5B,"Garden District, Ryerson",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Pizza Place,Bubble Tea Shop,Tea Room,Lingerie Store
8,1,M9B,"Princess Gardens, Martin Grove, West Deane Par...",Bank,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
9,1,M1C,"Highland Creek, Rouge Hill, Port Union",Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


##### Cluster 3: Cluster Label 2

In [56]:
Toronto_top.loc[Toronto_top[' Cluster labels']==2,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,2,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store
1,2,M6B,Glencairn,Park,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,2,M6E,Caledonia-Fairbanks,Park,Fast Food Restaurant,Market,Pharmacy,Women's Store,Grocery Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner
3,2,M4J,East Toronto,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,2,M3K,"Downsview East, CFB Toronto",Park,Airport,Other Repair Shop,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
5,2,M6L,"Downsview, Upwood Park, North Park",Park,Deli / Bodega,Bakery,Construction & Landscaping,Basketball Court,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
6,2,M4N,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,2,M9N,Weston,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
8,2,M2P,York Mills West,Park,Bank,Convenience Store,Bar,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,2,M5P,"Forest Hill North, Forest Hill West",Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


###### We observe that most of the neighborhoods in the Cluster have Park, Yoga Studio, Doner Restaurant, Donut Shop, Discount Store, Dog Run in top 10 venues

##### Cluster 4: Cluster label 3

In [57]:
Toronto_top.loc[Toronto_top[' Cluster labels']==3,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,3,M9M,"Humberlea, Emery",Baseball Field,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Dim Sum Restaurant
1,3,M8Y,"Humber Bay, Mimico NE, Kingsway Park South Eas...",Baseball Field,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Dim Sum Restaurant


###### We observe that the neighborhoods have same top 10 venues with same ranking

##### Cluster 5: Cluster label 4

In [58]:
Toronto_top.loc[Toronto_top[' Cluster labels']==4,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,4,M2L,"Silver Hills, York Mills",Cafeteria,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


#### We observe that maximum number of neighborhoods are in Cluster 2: Cluster label 1 